In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
#loading train set labels, test set labels and output of the LSTM layer of the probe model
df_train = pd.read_csv('../input/sentlen/train_labels.csv')
df_test_labels = pd.read_csv('../input/sentlen/test_labels.csv')
df_test = pd.read_csv('../input/sentlen/x_test.csv')

In [3]:
df_test

,test_sentences
0,0.017241
1,0.054898
2,0.071530
3,0.000000
4,0.000000
...,...
1499995,0.048950
1499996,0.000000
1499997,0.000000
1499998,0.000000


In [4]:
task_train_labels = df_train['train_labels'].to_numpy()
task_test_labels = df_test_labels['test_labels'].to_numpy()

In [5]:
task_train_labels = task_train_labels.reshape(-1,1)
task_test_labels = task_test_labels.reshape(-1,1)

In [6]:
#initialising the model


# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
        random_input = tf.keras.layers.Input(shape=(300))

        r = tf.keras.layers.Dense(300, activation = 'relu')(random_input)
        r = tf.keras.layers.Dropout(0.3)(r)
        r = tf.keras.layers.Dense(300, activation = 'relu')(r)
        r = tf.keras.layers.Dropout(0.3)(r)
        outputs = tf.keras.layers.Dense(6, activation = 'softmax')(r)

        random_model = tf.keras.Model(inputs = random_input, outputs = outputs)
        random_model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])   

In [7]:
#generating random word representations which will be used to train the model. Performance of the model when it is fed this random array will act as a baseline.
x_random = np.random.rand(60000,300)

In [8]:
random_model.fit(x_random,task_train_labels, epochs = 25, batch_size = 1024)

Epoch 1/25
59/59 [==============================] - 4s 20ms/step - loss: 1.8625 - accuracy: 0.1676
Epoch 2/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7942 - accuracy: 0.1694
Epoch 3/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7927 - accuracy: 0.1748
Epoch 4/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7926 - accuracy: 0.1697
Epoch 5/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7923 - accuracy: 0.1721
Epoch 6/25
59/59 [==============================] - 1s 14ms/step - loss: 1.7915 - accuracy: 0.1766
Epoch 7/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7921 - accuracy: 0.1757
Epoch 8/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7910 - accuracy: 0.1796
Epoch 9/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7901 - accuracy: 0.1763
Epoch 10/25
59/59 [==============================] - 1s 13ms/step - loss: 1.7897 - accuracy: 0.1796
Epoch 11/

In [9]:
x_random.shape,task_train_labels.shape

((60000, 300), (60000, 1))

In [10]:
x_test = df_test['test_sentences'].to_numpy()
x_test.shape

(1500000,)

In [11]:
x_test = x_test.reshape(5000,300)
x_test.shape

(5000, 300)

In [12]:
#evaluating this model, x_test is the actual output of the LSTM layer of the probe model which was saved at the last in the probe_model notebook
random_model.evaluate(x_test,task_test_labels)

157/157 [==============================] - 4s 15ms/step - loss: 1.7950 - accuracy: 0.1578


[1.7949886322021484, 0.15779998898506165]